In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, recall_score, precision_score
from scipy.stats import ranksums

from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.sampling.rnd import Sampling
from pymoo.operators.crossover.hux import HUX
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
from pymoo.optimize import minimize

from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch

from joblib import Parallel, delayed
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
import os
from scipy.stats.mstats import gmean

import wandb

with open('data.pickle', 'rb') as fh:
	data_mapper = pickle.load(fh)

In [2]:
from imblearn.over_sampling import (
    SMOTE,
    ADASYN,
    BorderlineSMOTE,
)
from imblearn.combine import (
    SMOTETomek,
    SMOTEENN
)

In [3]:
class AUC_Filter(Problem):
	population_size = 100
	n_neighbours = 5
	log_every = 5
	def __init__(self, X_train, y_train, X_val, y_val, X_test, Y_test, logger=None):
		
		self.generation_number = 0

		self.X_train = X_train
		self.y_train = y_train
		self.X_val = X_val
		self.y_val = y_val
		self.X_TEST = X_test
		self.Y_TEST = Y_test
		self.logger = logger

		self.training_data = X_train
		self.n_instances = X_train.shape[0]
		
		super().__init__(
			n_var=self.n_instances,
			n_obj=2,               
			n_constr=0,            
			xl=0,                  
			xu=1,                  
			type_var=np.bool_,     
		)

	def _evaluate(self, x, out, *args, **kwargs):
		
		values = []
		num_samples = []
		for instance in x:
			inverse_AUC = 1
			if np.sum(instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(
					n_neighbors=AUC_Optimizer.n_neighbours
				)
				model.fit(
					self.X_train[instance], 
					self.y_train[instance]
				)

				y_pred = model.predict(self.X_val)
				inverse_AUC = 1 - roc_auc_score(self.y_val, y_pred)
			
			num_samples.append(np.sum(instance))
			values.append(inverse_AUC)
		F = np.column_stack([values, num_samples])
		self.generation_number += 1
		
		if self.logger is not None and self.generation_number % AUC_Optimizer.log_every == 0:
			validation_aucs = []
			test_aucs = []
			pareto_indices = NonDominatedSorting().do(F, only_non_dominated_front=True)
			
			for idx in pareto_indices:
				instance = x[idx]

				if np.sum(instance) >= AUC_Optimizer.n_neighbours:
					model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
					model.fit(
						self.X_train[instance], 
						self.y_train[instance]
					)
					y_pred = model.predict(self.X_val)
					validation_aucs.append(roc_auc_score(self.y_val, y_pred))
					y_pred = model.predict(self.X_TEST)
					test_aucs.append(roc_auc_score(self.Y_TEST, y_pred))
				else:
					validation_aucs.append(0)
					test_aucs.append(0)
					
			validation_idx = np.argmax(validation_aucs)
			test_idx = np.argmax(test_aucs)	
			x_ideal_validation_instance = self.X_train[x[pareto_indices[validation_idx]]]
			y_ideal_validation_instance = self.y_train[x[pareto_indices[validation_idx]]]
			x_ideal_test_instance = self.X_train[x[pareto_indices[test_idx]]]
			y_ideal_test_instance = self.y_train[x[pareto_indices[test_idx]]]

			if len(x_ideal_validation_instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(
					x_ideal_validation_instance, 
					y_ideal_validation_instance
				)

				y_pred = model.predict(self.X_val)
				optimized_validation_auc = roc_auc_score(self.y_val, y_pred)
				
				y_pred = model.predict(self.X_TEST)
				optimized_test_auc = roc_auc_score(self.Y_TEST, y_pred)
			else:
				optimized_validation_auc = 0
				optimized_test_auc = 0

			# Calculate metrics using ideal instance w.r.t test AUC
			if len(x_ideal_test_instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(
					x_ideal_test_instance, 
					y_ideal_test_instance
				)
				
				y_pred = model.predict(self.X_TEST)
				ideal_test_auc = roc_auc_score(self.Y_TEST, y_pred)
			else:
				ideal_test_auc

			self.logger.log({
				"validation/optimized_AUC": optimized_validation_auc,
				"test/optimized_AUC": optimized_test_auc,
				"test/ideal_AUC": ideal_test_auc,
			})

		out["F"] = F

class AUC_Optimizer(Problem):
	population_size = 100
	n_neighbours = 5
	log_every = 5
	def __init__(self, X_train, y_train, X_val, y_val, X_test, Y_test, logger=None):
		
		self.generation_number = 0

		self.X_train = X_train
		self.y_train = y_train
		self.X_val = X_val
		self.y_val = y_val
		self.X_TEST = X_test
		self.Y_TEST = Y_test
		self.logger = logger

		self.training_data = X_train
		self.n_instances = X_train.shape[0]
		
		super().__init__(
			n_var=self.n_instances,
			n_obj=1,               
			n_constr=0,            
			xl=0,                  
			xu=1,                  
			type_var=np.bool_,     
		)

	def _evaluate(self, x, out, *args, **kwargs):
		
		values = []
		for instance in x:
			inverse_AUC = 1
			if np.sum(instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(
					n_neighbors=AUC_Optimizer.n_neighbours
				)
				model.fit(
					self.X_train[instance], 
					self.y_train[instance]
				)

				y_pred = model.predict(self.X_val)
				inverse_AUC = 1 - roc_auc_score(self.y_val, y_pred)
			
			values.append(inverse_AUC)
		F = np.column_stack([values])
		self.generation_number += 1
		
		if self.logger is not None and self.generation_number % AUC_Optimizer.log_every == 0:
			validation_aucs = []
			test_aucs = []
			pareto_indices = NonDominatedSorting().do(F, only_non_dominated_front=True)
			
			for idx in pareto_indices:
				instance = x[idx]

				if np.sum(instance) >= AUC_Optimizer.n_neighbours:
					model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
					model.fit(
						self.X_train[instance], 
						self.y_train[instance]
					)
					y_pred = model.predict(self.X_val)
					validation_aucs.append(roc_auc_score(self.y_val, y_pred))
					y_pred = model.predict(self.X_TEST)
					test_aucs.append(roc_auc_score(self.Y_TEST, y_pred))
				else:
					validation_aucs.append(0)
					test_aucs.append(0)
					
			validation_idx = np.argmax(validation_aucs)
			test_idx = np.argmax(test_aucs)	
			x_ideal_validation_instance = self.X_train[x[pareto_indices[validation_idx]]]
			y_ideal_validation_instance = self.y_train[x[pareto_indices[validation_idx]]]
			x_ideal_test_instance = self.X_train[x[pareto_indices[test_idx]]]
			y_ideal_test_instance = self.y_train[x[pareto_indices[test_idx]]]

			if len(x_ideal_validation_instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(
					x_ideal_validation_instance, 
					y_ideal_validation_instance
				)

				y_pred = model.predict(self.X_val)
				optimized_validation_auc = roc_auc_score(self.y_val, y_pred)
				
				y_pred = model.predict(self.X_TEST)
				optimized_test_auc = roc_auc_score(self.Y_TEST, y_pred)
			else:
				optimized_validation_auc = 0
				optimized_test_auc = 0

			# Calculate metrics using ideal instance w.r.t test AUC
			if len(x_ideal_test_instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(
					x_ideal_test_instance, 
					y_ideal_test_instance
				)
				
				y_pred = model.predict(self.X_TEST)
				ideal_test_auc = roc_auc_score(self.Y_TEST, y_pred)
			else:
				ideal_test_auc

			self.logger.log({
				"validation/optimized_AUC": optimized_validation_auc,
				"test/optimized_AUC": optimized_test_auc,
				"test/ideal_AUC": ideal_test_auc,
			})

		out["F"] = F

class DiverseCustomSampling(Sampling):
	def __init__(self):
		super().__init__()

	def _do(self, problem, n_samples, **kwargs):

		target_inclusions = np.random.randint(
			problem.n_var // 3,
			problem.n_var,
			n_samples
		)
		init_pops = []
		for target in target_inclusions:
			array = np.array([1]*target + [0]*(problem.n_var - target))
			np.random.shuffle(array)
			init_pops.append(array)
		init_pops = np.array(init_pops, dtype=np.bool)
		return init_pops

class ConditionalVAE(nn.Module):
	def __init__(self, input_dim, label_dim, hidden_dim, latent_dim):
		super(ConditionalVAE, self).__init__()
		self.fc1 = nn.Linear(input_dim + label_dim, hidden_dim)
		self.fc_mu = nn.Linear(hidden_dim, latent_dim)
		self.fc_logvar = nn.Linear(hidden_dim, latent_dim)
		self.fc2 = nn.Linear(latent_dim + label_dim, hidden_dim)
		self.fc3 = nn.Linear(hidden_dim, input_dim)

	def encode(self, x, y):
		# Concatenate input and label
		x = torch.cat([x, y], dim=1)
		h = torch.relu(self.fc1(x))
		return self.fc_mu(h), self.fc_logvar(h)

	def reparameterize(self, mu, logvar):
		std = torch.exp(0.5 * logvar)
		eps = torch.randn_like(std)
		return mu + eps * std

	def decode(self, z, y):
		# Concatenate latent vector and label
		z = torch.cat([z, y], dim=1)
		h = torch.relu(self.fc2(z))
		return self.fc3(h)

	def forward(self, x, y):
		mu, logvar = self.encode(x, y)
		z = self.reparameterize(mu, logvar)
		return self.decode(z, y), mu, logvar

class CustomDataset(Dataset):
	def __init__(self, x_synthetic, x_true):
		self.x = x_synthetic
		self.y = x_true
	def __len__(self):
		return self.x.shape[0]
	def __getitem__(self, ind):
		x = self.x[ind]
		y = self.y[ind]
		return x, y

def train(training_x, training_y, cvae, lr, epochs, batch_size, beta, logger=None):
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	train_set = CustomDataset(torch.from_numpy(training_x), torch.from_numpy(training_y))
	train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
	optimizer = optim.Adam(cvae.parameters(), lr=lr)
	
	for epoch in range(epochs):
		cvae.train()
		total_loss = 0
		for batch in train_loader:
			x_batch = batch[0].to(device).float()
			y_batch = batch[1].to(device).float().unsqueeze(1)
			
			recon, mu, logvar = cvae(x_batch, y_batch)
			recon_loss = nn.MSELoss()(recon, x_batch)
			kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
			loss = recon_loss + (kl_div*beta)
	
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()
			
			total_loss += loss.item()
		if logger is not None:
			logger.log({
				"training loss": total_loss / len(train_loader)
			})
		
	return cvae

def generate_synthetic_examples(x_samples, y_samples, sample_variance, cvae, num_samples=None):
	if num_samples is None:
		num_samples = len(x_samples)

	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

	cvae.eval()

	synthetic_features = []

	while len(synthetic_features) < num_samples:
		with torch.no_grad():    
			x = torch.from_numpy(x_samples).to(device).float()
			y = torch.from_numpy(y_samples).to(device).float().unsqueeze(1)
			mu, logvar = cvae.encode(x, y)
			z = cvae.reparameterize(mu, logvar)
		minority_latents = z.cpu().numpy()

		minority_latents[:,0] += np.random.normal(-sample_variance[0]/2, sample_variance[0]/2, len(minority_latents))
		minority_latents[:,1] += np.random.normal(-sample_variance[1]/2, sample_variance[1]/2, len(minority_latents))

		with torch.no_grad():    
			z = torch.from_numpy(minority_latents).to(device).float()
			label_dim = torch.from_numpy(y_samples).to(device).float().unsqueeze(1)
			synthetic_minority_samples = cvae.decode(z, label_dim)
	
		for sample in synthetic_minority_samples.cpu().numpy():
			if len(synthetic_features) < num_samples:
				synthetic_features.append(sample)
			else:
				break
			

	return np.array(synthetic_features)

def calculate_latent_dimension_variance(x, y, cvae):
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	with torch.no_grad():    
		x = torch.from_numpy(x).to(device).float()
		y = torch.from_numpy(y).to(device).float().unsqueeze(1)
		mu, logvar = cvae.encode(x, y)
		z = cvae.reparameterize(mu, logvar)
		total_latents = z.cpu().numpy()
		variance = np.var(total_latents, axis=0)
	
	return variance

def combine_sets(x1, y1, x2, y2):
	if len(x2) == 0:
		return x1, y1
	
	x = np.concatenate((x1, x2), axis=0)
	y = np.concatenate((y1, y2), axis=0)
	return x, y

def select_synthetic_survivors(result, x_train, y_train, x_validation, y_validation, real_samples, auc_selection_threshold):
	selected_samples = []
	sample_AUCs = []
	for individual in result.pop:

		if np.sum(individual.X) > AUC_Optimizer.n_neighbours:
			model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
			model.fit(x_train[individual.X], y_train[individual.X])
			y_pred = model.predict(x_validation)
			optimized_AUC = roc_auc_score(y_validation, y_pred)

			if optimized_AUC > auc_selection_threshold:
				for sample in x_train[individual.X]:
					for real_sample in real_samples:
						if np.all(sample == real_sample):
							break
					else:
						for saved_sample in selected_samples:
							if np.all(sample == saved_sample):
								break
						else:
							sample_AUCs.append(optimized_AUC)
							selected_samples.append(sample)
	
	idx = np.argsort(sample_AUCs)
	top_idx = idx[:10]
	arr = np.array(selected_samples)

	return arr[top_idx]

def split_info(data_key):
	segments = data_key.split('_')
	split_num = segments[0]
	dataset_name = '_'.join(segments[1:])
	return split_num, dataset_name

def calculate_IR(labels):
	counts = pd.DataFrame(labels).value_counts()
	return counts.max()/counts.min() 


In [4]:
data_key_iters = {}
for file in os.listdir("results"):
	segments = file.split(" ")
	data_key = segments[0]
	if data_key not in data_key_iters:
		data_key_iters[data_key] = []
	
	data_key_iters[data_key].append(file)

In [5]:
splits = pd.read_csv('data_splits.csv')
splits.columns

Index(['abalone-17_vs_7-8-9-10', 'abalone-20_vs_8-9-10', 'abalone-21_vs_8',
       'abalone-3_vs_11', 'abalone19', 'abalone9-18', 'cleveland-0_vs_4',
       'ecoli-0-1-4-7_vs_2-3-5-6', 'ecoli-0-1-4-7_vs_5-6',
       'ecoli-0-2-6-7_vs_3-5', 'ecoli-0-4-6_vs_5', 'ecoli-0-6-7_vs_3-5',
       'ecoli4', 'glass-0-6_vs_5', 'winequality-red-4',
       'winequality-red-8_vs_6-7', 'winequality-white-9_vs_4', 'wisconsin',
       'yeast-0-2-5-7-9_vs_3-6-8', 'yeast-2_vs_4'],
      dtype='object')

In [6]:
competitor = {}
for split_name in splits:
	for idx in range(31):

		results = {}
		data_key = f"{idx}_{split_name}"

		x_train = data_mapper[data_key]["x_train"]
		y_train = data_mapper[data_key]["y_train"]

		# x_validation = data_mapper[data_key]["x_validation"]
		# y_validation = data_mapper[data_key]["y_validation"]

		# x_comb = np.concatenate((x_train, x_validation), axis=0)
		# y_comb = np.concatenate((y_train, y_validation), axis=0)
		x_comb = x_train
		y_comb = y_train
		
		x_test = data_mapper[data_key]["x_test"]
		y_test = data_mapper[data_key]["y_test"]

		try:
			smote = SMOTE(sampling_strategy='minority') 
			x, y = smote.fit_resample(x_comb, y_comb)
			model = KNeighborsClassifier(n_neighbors=5)
			model.fit(x, y)
			y_pred = model.predict(x_test)
			results['SMOTE'] = roc_auc_score(y_test, y_pred)
		except:
			results['SMOTE'] = 0

		try:
			adasyn = ADASYN(sampling_strategy='minority')
			x, y = adasyn.fit_resample(x_comb, y_comb)
			model = KNeighborsClassifier(n_neighbors=5)
			model.fit(x, y)
			y_pred = model.predict(x_test)
			results['ADASYN'] = roc_auc_score(y_test, y_pred)
		except:
			results['ADASYN'] = 0
		
		try:
			blsmote = BorderlineSMOTE(sampling_strategy='minority', kind='borderline-1')
			x, y = blsmote.fit_resample(x_comb, y_comb)
			model = KNeighborsClassifier(n_neighbors=5)
			model.fit(x, y)
			y_pred = model.predict(x_test)
			results['Borderline SMOTE'] = roc_auc_score(y_test, y_pred)
		except:
			results['Borderline SMOTE'] = 0

		try:
			smt_tomek = SMOTETomek(sampling_strategy='auto')
			x, y = smt_tomek.fit_resample(x_comb, y_comb)
			model = KNeighborsClassifier(n_neighbors=5)
			model.fit(x, y)
			y_pred = model.predict(x_test)
			results['SMOTE Tomek'] = roc_auc_score(y_test, y_pred)
		except:
			results['SMOTE Tomek'] = 0
		
		try:
			smt_eenn = SMOTEENN(sampling_strategy='auto')
			x, y = smt_eenn.fit_resample(x_comb, y_comb)
			model = KNeighborsClassifier(n_neighbors=5)
			model.fit(x, y)
			y_pred = model.predict(x_test)
			results['SMOTE EENN'] = roc_auc_score(y_test, y_pred)
		except:
			results['SMOTE EENN'] = 0

		competitor[data_key] = results
competitor

{'0_abalone-17_vs_7-8-9-10': {'SMOTE': np.float64(0.7157894736842105),
  'ADASYN': np.float64(0.6783208020050125),
  'Borderline SMOTE': np.float64(0.7184210526315791),
  'SMOTE Tomek': np.float64(0.7114035087719298),
  'SMOTE EENN': np.float64(0.7418546365914787)},
 '1_abalone-17_vs_7-8-9-10': {'SMOTE': np.float64(0.7506265664160401),
  'ADASYN': np.float64(0.7140350877192982),
  'Borderline SMOTE': np.float64(0.7201754385964912),
  'SMOTE Tomek': np.float64(0.713157894736842),
  'SMOTE EENN': np.float64(0.775814536340852)},
 '2_abalone-17_vs_7-8-9-10': {'SMOTE': np.float64(0.712280701754386),
  'ADASYN': np.float64(0.7096491228070174),
  'Borderline SMOTE': np.float64(0.6783208020050125),
  'SMOTE Tomek': np.float64(0.6739348370927318),
  'SMOTE EENN': np.float64(0.7714285714285715)},
 '3_abalone-17_vs_7-8-9-10': {'SMOTE': np.float64(0.8419799498746867),
  'ADASYN': np.float64(0.8411027568922306),
  'Borderline SMOTE': np.float64(0.8176691729323309),
  'SMOTE Tomek': np.float64(0.879

In [ ]:
for dataset in splits.columns:

	synthetic_optimized_auc = []
	baseline_optimized_auc = []
	baseline_auc = []
	for data_key in splits[dataset]:
		x_train = data_mapper[data_key]['x_train'] 
		y_train = data_mapper[data_key]['y_train']
		x_validation = data_mapper[data_key]['x_validation'] 
		y_validation = data_mapper[data_key]['y_validation']
		x_test = data_mapper[data_key]['x_test'] 
		y_test = data_mapper[data_key]['y_test']

		max_validation_AUC = []
		result_by_idx = []

		model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		model.fit(x_train, y_train)
		y_pred = model.predict(x_test)
		baseline_auc.append(roc_auc_score(y_test, y_pred))
		# y_pred = model.predict(x_test)
		
		if data_key not in data_key_iters: continue
		
		for generation_file in data_key_iters[data_key]:
			with open(f"results/{generation_file}", "rb") as fh:
				result_dict = pickle.load(fh)

			validation_auc = []
			test_auc = []

			# for individual in result_dict['Result'].pop: >> No NDS
			# 	instance = individual.X
			for instance in result_dict['Result'].X:
				if np.sum(instance) >= AUC_Optimizer.n_neighbours:
					model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
					model.fit(result_dict['Base x_train'][instance], result_dict['Base y_train'][instance] )
					y_pred = model.predict(x_validation)
					validation_auc.append(roc_auc_score(y_validation, y_pred))
					# y_pred = model.predict(x_test)
					# test_auc.append(roc_auc_score(y_test, y_pred))
				else:
					validation_auc.append(0)
					# test_auc.append(0)

			# validation_idx = np.argmax(validation_auc)
			max_validation_AUC.append(np.max(validation_auc))
			result_by_idx.append(result_dict)
			# test_idx = np.argmax(test_auc)	
			# validation_ideal_x_train = result_dict['Base x_train'][result_dict['Result'].X[validation_idx]]
			# validation_ideal_y_train = result_dict['Base y_train'][result_dict['Result'].X[validation_idx]]
			# test_ideal_x_train = result_dict['Base x_train'][result_dict['Result'].X[test_idx]]
			# test_ideal_y_train = result_dict['Base y_train'][result_dict['Result'].X[test_idx]]

		best_generation_idx = np.argmax(max_validation_AUC)
		synthetic_optimized_auc.append(np.max(max_validation_AUC))
		best_result = result_by_idx[best_generation_idx]
		
		test_auc = []
		for instance in best_result['Result'].X:
			if np.sum(instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(best_result['Base x_train'][instance], best_result['Base y_train'][instance] )
				y_pred = model.predict(x_test)
				test_auc.append(roc_auc_score(y_test, y_pred))
			else:
				test_auc.append(0)
		test_idx = np.argmax(test_auc)

		test_ideal_x_train = best_result['Base x_train'][best_result['Result'].X[test_idx]]
		test_ideal_y_train = best_result['Base y_train'][best_result['Result'].X[test_idx]]

		model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		model.fit(test_ideal_x_train, test_ideal_y_train)
		y_pred = model.predict(x_test)
		synthetic_optimized_auc.append(roc_auc_score(y_test, y_pred))

		#########################################
		#
		##########################################

		problem = AUC_Filter(
			x_train, y_train, 
			x_validation, y_validation,
			X_test=x_test, Y_test=y_test,
		)
		algorithm = NSGA2(pop_size=AUC_Optimizer.population_size, sampling=DiverseCustomSampling(), crossover=HUX(), mutation=BitflipMutation(), eliminate_duplicates=True)
		result = minimize(problem, algorithm, ('n_gen', AUC_Optimizer.population_size), save_history=False)
		
		for instance in result.X:
			if np.sum(instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(x_train[instance], y_train[instance] )
				y_pred = model.predict(x_test)
				test_auc.append(roc_auc_score(y_test, y_pred))
			else:
				test_auc.append(0)

		baseline_optimized_auc.append(np.max(test_auc))
	print(f"\n{dataset}: ")
	print(f"  PVAL:             {round(ranksums(baseline_optimized_auc, synthetic_optimized_auc).pvalue, 4)}  {"TRUE" if round(ranksums(baseline_optimized_auc, synthetic_optimized_auc).pvalue, 4) < 0.05 else "FALSE"}")
	print(f"  Mean synthOve AUC {round(np.mean(synthetic_optimized_auc), 4)}")
	print(f"  Mean baseline AUC {round(np.mean(baseline_optimized_auc), 4)} ")
	print(f"  Mean noOptimi AUC {round(np.mean(baseline_auc), 4)}")
		
	


abalone-17_vs_7-8-9-10: 
  PVAL:             0.0022  TRUE
  Mean synthOve AUC 0.7619
  Mean baseline AUC 0.7016 
  Mean noOptimi AUC 0.5259

abalone-20_vs_8-9-10: 
  PVAL:             0.0381  TRUE
  Mean synthOve AUC 0.8021
  Mean baseline AUC 0.7542 
  Mean noOptimi AUC 0.5302

abalone-21_vs_8: 
  PVAL:             0.1157  FALSE
  Mean synthOve AUC 0.9162
  Mean baseline AUC 0.8859 
  Mean noOptimi AUC 0.6771

abalone-3_vs_11: 
  PVAL:             0.4509  FALSE
  Mean synthOve AUC 0.9991
  Mean baseline AUC 0.9987 
  Mean noOptimi AUC 1.0

abalone19: 
  PVAL:             0.6277  FALSE
  Mean synthOve AUC 0.504
  Mean baseline AUC 0.5 
  Mean noOptimi AUC 0.5

abalone9-18: 
  PVAL:             0.0009  TRUE
  Mean synthOve AUC 0.8669
  Mean baseline AUC 0.8044 
  Mean noOptimi AUC 0.5703

cleveland-0_vs_4: 
  PVAL:             0.1057  FALSE
  Mean synthOve AUC 0.9426
  Mean baseline AUC 0.9257 
  Mean noOptimi AUC 0.682

ecoli-0-1-4-7_vs_2-3-5-6: 
  PVAL:             0.0049  TRUE
  Mea

In [ ]:
records = []

for dataset in splits.columns:

	synthetic_optimized_auc = []
	nsga_2 = []
	smote = []
	adasyn = []
	borderline_smote = []
	smote_tomek = []
	smote_enn = []

	irs = []

	print(dataset)
	for data_key in splits[dataset]:
		x_train = data_mapper[data_key]['x_train'] 
		y_train = data_mapper[data_key]['y_train']
		x_validation = data_mapper[data_key]['x_validation'] 
		y_validation = data_mapper[data_key]['y_validation']
		x_test = data_mapper[data_key]['x_test'] 
		y_test = data_mapper[data_key]['y_test']

		counts = pd.DataFrame(y_train).value_counts()
		irs.append(counts.max()/counts.min() )
		max_validation_AUC = []
		result_by_idx = []

		if data_key not in data_key_iters: continue
		
		for generation_file in data_key_iters[data_key]:
			with open(f"results/{generation_file}", "rb") as fh:
				result_dict = pickle.load(fh)

			validation_auc = []
			test_auc = []

			for instance in result_dict['Result'].X:
				if np.sum(instance) >= AUC_Optimizer.n_neighbours:
					model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
					model.fit(result_dict['Base x_train'][instance], result_dict['Base y_train'][instance] )
					y_pred = model.predict(x_validation)
					validation_auc.append(roc_auc_score(y_validation, y_pred))
					# y_pred = model.predict(x_test)
					# test_auc.append(roc_auc_score(y_test, y_pred))
				else:
					validation_auc.append(0)
					# test_auc.append(0)

			# validation_idx = np.argmax(validation_auc)
			max_validation_AUC.append(np.max(validation_auc))
			result_by_idx.append(result_dict)
			# test_idx = np.argmax(test_auc)	
			# validation_ideal_x_train = result_dict['Base x_train'][result_dict['Result'].X[validation_idx]]
			# validation_ideal_y_train = result_dict['Base y_train'][result_dict['Result'].X[validation_idx]]
			# test_ideal_x_train = result_dict['Base x_train'][result_dict['Result'].X[test_idx]]
			# test_ideal_y_train = result_dict['Base y_train'][result_dict['Result'].X[test_idx]]

		best_generation_idx = np.argmax(max_validation_AUC)
		# synthetic_optimized_auc.append(np.max(max_validation_AUC))
		best_result = result_by_idx[best_generation_idx]
		
		# test_auc = []
		validation_auc = []
		for instance in best_result['Result'].X:
			if np.sum(instance) > AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(best_result['Base x_train'][instance], best_result['Base y_train'][instance] )
				# y_pred = model.predict(x_test)
				# test_auc.append(roc_auc_score(y_test, y_pred))
				y_pred = model.predict(x_validation)
				validation_auc.append(roc_auc_score(y_validation, y_pred))
			else:
				validation_auc.append(0)
		# test_idx = np.argmax(test_auc)
		validation_idx = np.argmax(validation_auc)
		
		# test_ideal_x_train = best_result['Base x_train'][best_result['Result'].X[test_idx]]
		# test_ideal_y_train = best_result['Base y_train'][best_result['Result'].X[test_idx]]
		validation_ideal_x_train = best_result['Base x_train'][best_result['Result'].X[validation_idx]]
		validation_ideal_y_train = best_result['Base y_train'][best_result['Result'].X[validation_idx]]

		model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		model.fit(validation_ideal_x_train, validation_ideal_y_train)
		y_pred = model.predict(x_test)
		synthetic_optimized_auc.append(roc_auc_score(y_test, y_pred))

		# model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		# model.fit(test_ideal_x_train, test_ideal_y_train)
		# y_pred = model.predict(x_test)
		# synthetic_optimized_auc.append(roc_auc_score(y_test, y_pred))

		#########################################
		#
		##########################################

		competitor_results = competitor[data_key]
		smote.append(competitor_results['SMOTE'])
		adasyn.append(competitor_results['ADASYN'])
		borderline_smote.append(competitor_results['Borderline SMOTE'])
		smote_tomek.append(competitor_results['SMOTE Tomek'])
		smote_enn.append(competitor_results['SMOTE EENN'])

		#########################################
		#
		##########################################
		
		# problem = AUC_Filter(
		# 	x_train, y_train, 
		# 	x_validation, y_validation,
		# 	X_test=x_test, Y_test=y_test,
		# )
		# algorithm = NSGA2(pop_size=AUC_Optimizer.population_size, sampling=DiverseCustomSampling(), crossover=HUX(), mutation=BitflipMutation(), eliminate_duplicates=True)
		# result = minimize(problem, algorithm, ('n_gen', AUC_Optimizer.population_size), save_history=False)
		# baseline_validation_auc = []
		# for instance in result.X:
		# 	if np.sum(instance) > AUC_Optimizer.n_neighbours:
		# 		model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		# 		model.fit(x_train[instance], y_train[instance] )
		# 		y_pred = model.predict(x_validation)
		# 		baseline_validation_auc.append(roc_auc_score(y_validation, y_pred))
		# 	else:
		# 		baseline_validation_auc.append(0)

		# best_wrt_validation = np.argmax(baseline_validation_auc)

		# if np.sum(result.X[best_wrt_validation]) > AUC_Optimizer.n_neighbours:
		# 	model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		# 	model.fit(x_train[result.X[best_wrt_validation]], y_train[result.X[best_wrt_validation]])
		# 	y_pred = model.predict(x_test)
		# 	nsga_2.append(roc_auc_score(y_test, y_pred))
		# else:
		# 	nsga_2.append(0)

	smote_wx = ranksums(smote, synthetic_optimized_auc).pvalue
	adasyn_wx = ranksums(adasyn, synthetic_optimized_auc).pvalue
	borderline_wx = ranksums(borderline_smote, synthetic_optimized_auc).pvalue
	tomek_wx = ranksums(smote_tomek, synthetic_optimized_auc).pvalue
	enn_wx = ranksums(smote_enn, synthetic_optimized_auc).pvalue
	nsga_wx = ranksums(nsga_2, synthetic_optimized_auc).pvalue

	me_auc = np.mean(synthetic_optimized_auc)
	smote_auc = np.mean(smote)
	adasyn_auc = np.mean(adasyn)
	borderline_auc = np.mean(borderline_smote)
	tomek_auc = np.mean(smote_tomek)
	enn_auc = np.mean(smote_enn)
	nsga_auc = np.mean(nsga_2)
	
	rankings = [me_auc, smote_auc, adasyn_auc, borderline_auc, tomek_auc, enn_auc, nsga_auc]
	ranking_titles = ['cVAE', 'SMOTE', 'ADASYN', 'BORDERLINE', 'TOMEK', 'ENN', 'NSGA-2']

	
	sorted_titles = pd.DataFrame(ranking_titles).iloc[np.argsort(rankings)]

	records.append({
		"IR": round(np.mean(irs), 2),
		"Dataset": dataset,
		"ME VS SMOTE": smote_wx,
		"ME VS ADASYN": adasyn_wx,
		"ME VS Borderline Smote": borderline_wx,
		"ME VS SMOTE Tomek": tomek_wx,
		"ME VS SMOTE ENN": enn_wx,
		"ME VS NSGA": nsga_wx,
		"cVAE": me_auc,
		"SMOTE": smote_auc,
		"ADASYN": adasyn_auc,
		"BORDERLINE": borderline_auc,
		"TOMEK": tomek_auc,
		"ENN": enn_auc,
		"NSGA-2": nsga_auc,
		"Ranked AUCs": np.sort(rankings),
		"Ranked titles": sorted_titles,
	})

	print(records[-1])
tt = pd.DataFrame.from_records(records)
# tt.head()
tt.to_excel('NEWEST.xlsx',index=False)
		
	

abalone-17_vs_7-8-9-10
{'IR': np.float64(39.31), 'Dataset': 'abalone-17_vs_7-8-9-10', 'ME VS SMOTE': np.float64(3.564292259657079e-10), 'ME VS ADASYN': np.float64(7.981384840368783e-10), 'ME VS Borderline Smote': np.float64(5.138983899513966e-09), 'ME VS SMOTE Tomek': np.float64(5.109952508030462e-10), 'ME VS SMOTE ENN': np.float64(1.566933968351836e-10), 'ME VS NSGA': np.float64(0.06311624676030969), 'cVAE': np.float64(0.6361063950198076), 'SMOTE': np.float64(0.7726210688010348), 'ADASYN': np.float64(0.7725563909774436), 'BORDERLINE': np.float64(0.7638289271566013), 'TOMEK': np.float64(0.7796466973886327), 'ENN': np.float64(0.8092246745896999), 'NSGA-2': np.float64(0.6039776861508611), 'Ranked AUCs': array([0.60397769, 0.6361064 , 0.76382893, 0.77255639, 0.77262107,
       0.7796467 , 0.80922467]), 'Ranked titles':             0
6      NSGA-2
0        cVAE
3  BORDERLINE
2      ADASYN
1       SMOTE
4       TOMEK
5         ENN}
abalone-20_vs_8-9-10
{'IR': np.float64(72.69), 'Dataset': '

In [ ]:
records = []

for dataset in splits.columns:

	synthetic_optimized_auc = []
	
	smote = []
	adasyn = []
	borderline_smote = []
	smote_tomek = []
	smote_enn = []

	irs = []

	for data_key in splits[dataset]:
		x_train = data_mapper[data_key]['x_train'] 
		y_train = data_mapper[data_key]['y_train']
		x_validation = data_mapper[data_key]['x_validation'] 
		y_validation = data_mapper[data_key]['y_validation']
		x_test = data_mapper[data_key]['x_test'] 
		y_test = data_mapper[data_key]['y_test']

		counts = pd.DataFrame(y_train).value_counts()
		irs.append(counts.max()/counts.min() )
		max_validation_AUC = []
		result_by_idx = []

		# model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		# model.fit(x_train, y_train)
		# y_pred = model.predict(x_test)
		# baseline_auc.append(roc_auc_score(y_test, y_pred))
		# y_pred = model.predict(x_test)
		
		if data_key not in data_key_iters: continue
		
		for generation_file in data_key_iters[data_key]:
			with open(f"results/{generation_file}", "rb") as fh:
				result_dict = pickle.load(fh)

			validation_auc = []
			test_auc = []

			# for individual in result_dict['Result'].pop: >> No NDS
			# 	instance = individual.X
			for instance in result_dict['Result'].X:
				if np.sum(instance) >= AUC_Optimizer.n_neighbours:
					model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
					model.fit(result_dict['Base x_train'][instance], result_dict['Base y_train'][instance] )
					y_pred = model.predict(x_validation)
					validation_auc.append(roc_auc_score(y_validation, y_pred))
					# y_pred = model.predict(x_test)
					# test_auc.append(roc_auc_score(y_test, y_pred))
				else:
					validation_auc.append(0)
					# test_auc.append(0)

			# validation_idx = np.argmax(validation_auc)
			max_validation_AUC.append(np.max(validation_auc))
			result_by_idx.append(result_dict)
			# test_idx = np.argmax(test_auc)	
			# validation_ideal_x_train = result_dict['Base x_train'][result_dict['Result'].X[validation_idx]]
			# validation_ideal_y_train = result_dict['Base y_train'][result_dict['Result'].X[validation_idx]]
			# test_ideal_x_train = result_dict['Base x_train'][result_dict['Result'].X[test_idx]]
			# test_ideal_y_train = result_dict['Base y_train'][result_dict['Result'].X[test_idx]]

		best_generation_idx = np.argmax(max_validation_AUC)
		# synthetic_optimized_auc.append(np.max(max_validation_AUC))
		best_result = result_by_idx[best_generation_idx]
		
		test_auc = []
		for instance in best_result['Result'].X:
			if np.sum(instance) >= AUC_Optimizer.n_neighbours:
				model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
				model.fit(best_result['Base x_train'][instance], best_result['Base y_train'][instance] )
				y_pred = model.predict(x_test)
				test_auc.append(roc_auc_score(y_test, y_pred))
			else:
				test_auc.append(0)
		test_idx = np.argmax(test_auc)

		test_ideal_x_train = best_result['Base x_train'][best_result['Result'].X[test_idx]]
		test_ideal_y_train = best_result['Base y_train'][best_result['Result'].X[test_idx]]

		model = KNeighborsClassifier(n_neighbors=AUC_Optimizer.n_neighbours)
		model.fit(test_ideal_x_train, test_ideal_y_train)
		y_pred = model.predict(x_test)
		synthetic_optimized_auc.append(roc_auc_score(y_test, y_pred))

		#########################################
		#
		##########################################

		competitor_results = competitor[data_key]
		smote.append(competitor_results['SMOTE'])
		adasyn.append(competitor_results['ADASYN'])
		borderline_smote.append(competitor_results['Borderline SMOTE'])
		smote_tomek.append(competitor_results['SMOTE Tomek'])
		smote_enn.append(competitor_results['SMOTE EENN'])
	
	smote_wx = ranksums(smote, synthetic_optimized_auc).pvalue
	adasyn_wx = ranksums(adasyn, synthetic_optimized_auc).pvalue
	borderline_wx = ranksums(borderline_smote, synthetic_optimized_auc).pvalue
	tomek_wx = ranksums(smote_tomek, synthetic_optimized_auc).pvalue
	enn_wx = ranksums(smote_enn, synthetic_optimized_auc).pvalue

	me_auc = np.mean(synthetic_optimized_auc)
	smote_auc = np.mean(smote)
	adasyn_auc = np.mean(adasyn)
	borderline_auc = np.mean(borderline_smote)
	tomek_auc = np.mean(smote_tomek)
	enn_auc = np.mean(smote_enn)
	
	rankings = [me_auc, smote_auc, adasyn_auc, borderline_auc, tomek_auc, enn_auc]
	ranking_titles = ['cVAE', 'SMOTE', 'ADASYN', 'BORDERLINE', 'TOMEK', 'ENN']

	
	sorted_titles = pd.DataFrame(ranking_titles).iloc[np.argsort(rankings)]

	records.append({
		"IR": round(np.mean(irs), 2),
		"Dataset": dataset,
		"ME VS SMOTE": smote_wx,
		"ME VS ADASYN": adasyn_wx,
		"ME VS Borderline Smote": borderline_wx,
		"ME VS SMOTE Tomek": tomek_wx,
		"ME VS SMOTE ENN": enn_wx,
		"cVAE": me_auc,
		"SMOTE": smote_auc,
		"ADASYN": adasyn_auc,
		"BORDERLINE": borderline_auc,
		"TOMEK": tomek_auc,
		"ENN": enn_auc,
		"Ranked AUCs": np.sort(rankings),
		"Ranked titles": sorted_titles,
	})

tt = pd.DataFrame.from_records(records)
tt.head()

		
	